<a href="https://colab.research.google.com/github/DataScienceSolutions/GNN/blob/main/Intro_to_graph_neural_networks_(ML_Tech_Talks).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to graph neural networks (ML Tech Talks)

https://www.youtube.com/watch?v=8owQBFAHw7E&t=1937s

In [ ]:
!pip install numpy==1.18.5
!pip install tensorflow==2.3.0
!pip install spektral==0.6.2

import numpy as np
import tensorflow as tf
import spektral

In [ ]:
# Runtime / Change runtime type / Hardware = GPU
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6675431160962305996, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 18413698269756290677
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1517182659890164298
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14648653952
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16905184592759017185
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [ ]:
adj, features, labels, train_mask, val_mask, test_mask = spektral.datasets.citation.load_data(dataset_name='cora')

features = features.todense()
adj = adj.todense() + np.eye(adj.shape[0])
features = features.astype('float32')
adj = adj.astype('float32')

print(features.shape)
print(adj.shape)
print(labels.shape)

print(np.sum(train_mask))
print(np.sum(val_mask))
print(np.sum(test_mask))

Loading cora dataset
Pre-processing node features
(2708, 1433)
(2708, 2708)
(2708, 7)
140
500
1000


In [ ]:
def masked_softmax_cross_entropy(logits, labels, mask):
  loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)
  mask = tf.cast(mask, dtype=tf.float32)
  mask /= tf.reduce_mean(mask)
  loss *= mask
  return tf.reduce_mean(loss)

def masked_accuracy(logits, labels, mask):
  correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
  accuracy_all = tf.cast(correct_prediction, tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  mask /= tf.reduce_mean(mask)
  accuracy_all *= mask
  return tf.reduce_mean(accuracy_all)

In [ ]:
def gnn(fts, adj, transform, activation):
  seq_fts = transform(fts)
  ret_fts = tf.matmul(adj, seq_fts)
  return activation(ret_fts)

In [ ]:
def train_cora(fts, adj, gnn_fn, units, epochs, lr):
  lyr_1 = tf.keras.layers.Dense(units)
  lyr_2 = tf.keras.layers.Dense(7)

  def cora_gnn(fts, adj):
    hidden = gnn_fn(fts, adj, lyr_1, tf.nn.relu)
    logits = gnn_fn(hidden, adj, lyr_2, tf.identity)
    return logits

  optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

  best_accuracy = 0.0
  for ep in range(epochs +1):
    with tf.GradientTape() as t:
      logits = cora_gnn(fts, adj)
      loss = masked_softmax_cross_entropy(logits, labels, train_mask)

    variables = t.watched_variables()
    grads = t.gradient(loss, variables)
    optimizer.apply_gradients(zip(grads, variables))

    logits = cora_gnn(fts, adj)
    val_accuracy = masked_accuracy(logits, labels, val_mask)
    test_accuracy = masked_accuracy(logits, labels, test_mask)
    
    if val_accuracy > best_accuracy:
      best_accuracy = val_accuracy
      print('Epoch', ep, '| Training loss:', loss.numpy(), '| Val accuracy:', val_accuracy.numpy(), '|Test accuracy:', test_accuracy.numpy())


In [ ]:
train_cora(features, adj, gnn, 32, 200, 0.01)

Epoch 0 | Training loss: 1.9734956 | Val accuracy: 0.15599999 |Test accuracy: 0.144
Epoch 1 | Training loss: 2.3207216 | Val accuracy: 0.29599997 |Test accuracy: 0.30699998
Epoch 2 | Training loss: 1.6255368 | Val accuracy: 0.426 |Test accuracy: 0.44399998
Epoch 3 | Training loss: 1.5444814 | Val accuracy: 0.47 |Test accuracy: 0.509
Epoch 4 | Training loss: 1.4866936 | Val accuracy: 0.502 |Test accuracy: 0.5309999
Epoch 5 | Training loss: 1.3794113 | Val accuracy: 0.64599997 |Test accuracy: 0.64799994
Epoch 6 | Training loss: 1.2298715 | Val accuracy: 0.678 |Test accuracy: 0.71299994
Epoch 7 | Training loss: 1.0824136 | Val accuracy: 0.71799994 |Test accuracy: 0.7289999
Epoch 9 | Training loss: 0.9246315 | Val accuracy: 0.72599995 |Test accuracy: 0.7239999
Epoch 10 | Training loss: 0.85069346 | Val accuracy: 0.72999996 |Test accuracy: 0.73099995
Epoch 11 | Training loss: 0.78055537 | Val accuracy: 0.73399997 |Test accuracy: 0.7439999
Epoch 19 | Training loss: 0.39374086 | Val accuracy:

In [ ]:
# NN w/o graph structure (identity for adjacency matrix)
train_cora(features, tf.eye(adj.shape[0]), gnn, 32, 200, 0.01)

Epoch 0 | Training loss: 1.9470893 | Val accuracy: 0.19199999 |Test accuracy: 0.19099998
Epoch 1 | Training loss: 1.9360133 | Val accuracy: 0.24599999 |Test accuracy: 0.27299997
Epoch 2 | Training loss: 1.9204361 | Val accuracy: 0.36999997 |Test accuracy: 0.378
Epoch 5 | Training loss: 1.8436835 | Val accuracy: 0.38399997 |Test accuracy: 0.38199997
Epoch 6 | Training loss: 1.8114039 | Val accuracy: 0.41200003 |Test accuracy: 0.40599996
Epoch 7 | Training loss: 1.7756189 | Val accuracy: 0.44799998 |Test accuracy: 0.437
Epoch 8 | Training loss: 1.7368519 | Val accuracy: 0.45999998 |Test accuracy: 0.46299994
Epoch 9 | Training loss: 1.695024 | Val accuracy: 0.478 |Test accuracy: 0.46999997
Epoch 11 | Training loss: 1.6025627 | Val accuracy: 0.48799992 |Test accuracy: 0.48799998
Epoch 12 | Training loss: 1.5522933 | Val accuracy: 0.49799994 |Test accuracy: 0.48999992
Epoch 13 | Training loss: 1.4994979 | Val accuracy: 0.50399995 |Test accuracy: 0.49199992
Epoch 14 | Training loss: 1.444385